# <center> Linear Regression Model
**Summary of Actions**
* A simlpe linear regression model

### Import Preliminaries

In [159]:
%matplotlib inline

# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


# Import data
train_df = pd.DataFrame(pd.read_csv(
    'https://www.dropbox.com/s/8ftdd4tejbmesy4/featured_train_df.csv?dl=1', index_col='Id'))
test_df = pd.DataFrame(pd.read_csv(
    'https://www.dropbox.com/s/h672z8790ehkxuj/featured_test_df.csv?dl=1', index_col='Id'))
test_df.drop('SalePrice', axis=1, inplace=True)

# Set pandas options
pd.set_option('precision', 1)
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)

dfs = [train_df, test_df]

### DataFrame to Values

In [160]:
X = train_df.loc[:, train_df.columns != 'SalePrice'].copy()
y = train_df.SalePrice
X_test = test_df.values

### Fitting the Model 

In [161]:
model = LinearRegression()
model.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Model Predictions

In [162]:
# create predictions dataframe
predictions = pd.DataFrame(test_df.index)
saleprice = pd.DataFrame(model.predict(X_test),
                          columns=['SalePrice'])
predictions = pd.concat([predictions, saleprice],axis=1)
predictions.head()

,Id,SalePrice
0,1461,106034.5
1,1462,158728.1
2,1463,165980.7
3,1464,184264.8
4,1465,187973.7


### Model Crossvalidation

In [163]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cross_validation = KFold(10, random_state=7)
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cross_validation, n_jobs=1)

print ('Folds: %i, Mean Squared Error: %.2f std: %.2f' 
% (len(scores),np.mean(np.abs(scores)),np.std(scores)))

Folds: 10, Mean Squared Error: 1323093483.21 std: 1055874954.55


### Residual Plot

### Export Results

In [168]:
predictions.to_csv('Submissions/lr_predicition.csv', index=False)

### Accuracy Scores

Feature Enginnering - MSE: 1323219735.68, std: 1055098315.40

### Kaggle Scores